In [22]:
import numpy as np
import sys
import os
import neuro
import risp
import random
import time

import json

from encode import Chromagram
from encode import Encoder
from data_loader import data_loader
import risp
import neuro
import eons
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score


1.) Innitialize the neuroprocessor

In [23]:
risp_config = {
  "leak_mode": "all",
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "max_delay": 5,
  "discrete": False
}

proc = risp.Processor(risp_config) # RISP processor

net = neuro.Network() # Neuro network
net.set_properties(proc.get_network_properties()) # Set network properties


2.) Load the data

In [3]:
# First, split the data into training+validation and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(data_loader().chroma_files, data_loader().numerical_label_roots, test_size=0.2, random_state=42)

# Now, split the training+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# One-hot encode the labels
OneHot = OneHotEncoder(sparse_output=False)
y_train_encoded = OneHot.fit_transform(np.array(y_train).reshape(-1, 1))
y_val_encoded = OneHot.transform(np.array(y_val).reshape(-1, 1))
y_test_encoded = OneHot.transform(np.array(y_test).reshape(-1, 1))

3.) create template for eons

In [4]:
n_inputs = 12
n_hidden = 1200
n_outputs = 12
n_neurons = n_inputs + n_hidden + n_outputs
n_synapses = 8000


moa = neuro.MOA()
moa.seed(42)

In [5]:
def create_neuron(neuron_id, net, moa):
    neuron = net.add_node(neuron_id)
    net.randomize_node_properties(moa, neuron)
    return neuron

In [6]:
for i in range(n_inputs):
    neuron = create_neuron(i, net, moa)
    neuron.set("Threshold",0.75)
    net.add_input(i)
    
for i in range(n_outputs):
    neuron = create_neuron(i+n_inputs, net, moa)
    neuron.set("Threshold",0.75)
    net.add_output(i)
    
for i in range(n_hidden):
    neuron = create_neuron(i+n_inputs+n_outputs, net, moa)

In [7]:
print("Neuron 0's input ID:", net.get_node(0).input_id)
print("Neuron 1's input ID:", net.get_node(1).input_id)

Neuron 0's input ID: 0
Neuron 1's input ID: 1


In [8]:
for i in range(n_synapses):
    source = random.randint(0,n_neurons-1)
    dest = random.randint(0,n_neurons-1)
    synapse = net.add_or_get_edge(source, dest)
    net.randomize_edge_properties(moa, synapse)

4.) set up the eons

In [29]:
eo_params =  {
    "starting_nodes": 60,
    "starting_edges": 41,
    "merge_rate": 0.05,
    "population_size": 100,
    "multi_edges": 0.05,
    "crossover_rate": 0.5,
    "mutation_rate": 0.9,
    "selection_type": "tournament",
    "tournament_size_factor": 0.33,
    "tournament_best_net_factor": 0.9,
    "random_factor": 0,
    "num_mutations": 8,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.8, "Delay": 0.3 },
    "num_best": 2,

    "add_node_rate": 0.85,
    "delete_node_rate": 0.25,
    "add_edge_rate": 0.85,
    "delete_edge_rate": 0.25,
    "node_params_rate": 2.5,
    "edge_params_rate": 2.5,
    "net_params_rate" : 0
}


In [6]:
def read_network(fn):
    with open(fn, 'r') as f:
        s = f.read()
        j = json.loads(s)
        net = neuro.Network()
        net.from_json(j)
    return net

In [24]:
network3 = read_network('model_31.json')

In [25]:
print(network3.num_edges())
print(network3.num_nodes())

41
60


In [44]:
evolver = eons.EONS(eo_params)
evolver.set_template_network(network3)
pop = evolver.generate_population(eo_params,42)

In [42]:
def get_prediction(proc, x):
    # Load the chroma data from the npy file
    encoder=Encoder(x)
    
    proc.clear_activity()
    proc.apply_spikes(encoder.spikes)
    proc.run(encoder.time_steps * encoder.num_frames * 4) # you might adjust this duration based on your needs
    
    # Decoding the output to get the predicted label. You might need to adjust this
    predicted_index = proc.output_count_max(n_outputs)[0]
    # Convert index to one-hot encoded format
    one_hot_prediction = np.zeros(n_outputs)
    one_hot_prediction[predicted_index] = 1.0
    return one_hot_prediction 


In [13]:
def fitness(proc, net, X, y):
    proc.load_network(net)
    
    # Set up output tracking
    for i in range(n_outputs):
        proc.track_neuron_events(i)
    
    y_predict = [get_prediction(proc, x) for x in X]
    return accuracy_score(y_predict, y)


In [47]:
max_generations = 200
vals = []

for generation in range(max_generations):
    start_time = time.time()

    # Evaluate fitness of all networks in the population
    fitnesses = [fitness(proc, net.network, X_train, y_train_encoded) for net in pop.networks]
    
    # Track and print best fitness in the current generation
    best_fitness = max(fitnesses)
    vals.append(best_fitness)
    best_net = pop.networks[fitnesses.index(best_fitness)].network

    end_time = time.time()
    #elapsed time in format hh:mm:ss
    elapsed_time = time.strftime("%H:%M:%S", time.gmtime(end_time - start_time))
    total_time = time.strftime("%H:%M:%S", time.gmtime(end_time - start_time))

    print(f"Gen {generation + 1} Best Fitness: {best_fitness:.4f}, Max Nodes: {best_net.num_nodes()} Elapsed Time: {elapsed_time}" )
    
    # Produce the next generation based on the current population's fitness``
    pop = evolver.do_epoch(pop, fitnesses, eo_params)

# Optionally, you can evaluate and print the performance of the best network on the test set here.
print("Evolution finished!")
print("Best fitness:", best_net)

Gen 1 Best Fitness: 0.1279, Max Nodes: 72 Elapsed Time: 00:07:57
Gen 2 Best Fitness: 0.1627, Max Nodes: 73 Elapsed Time: 00:07:53
Gen 3 Best Fitness: 0.1866, Max Nodes: 73 Elapsed Time: 00:07:51
Gen 4 Best Fitness: 0.1924, Max Nodes: 74 Elapsed Time: 00:07:59
Gen 5 Best Fitness: 0.1924, Max Nodes: 76 Elapsed Time: 00:08:09
Gen 6 Best Fitness: 0.1941, Max Nodes: 77 Elapsed Time: 00:08:08
Gen 7 Best Fitness: 0.1941, Max Nodes: 77 Elapsed Time: 00:08:05
Gen 8 Best Fitness: 0.2061, Max Nodes: 76 Elapsed Time: 00:08:02
Gen 9 Best Fitness: 0.2072, Max Nodes: 77 Elapsed Time: 00:07:58
Gen 10 Best Fitness: 0.2078, Max Nodes: 77 Elapsed Time: 00:08:03
Gen 11 Best Fitness: 0.2083, Max Nodes: 76 Elapsed Time: 00:08:06
Gen 12 Best Fitness: 0.2083, Max Nodes: 77 Elapsed Time: 00:08:05
Gen 13 Best Fitness: 0.2089, Max Nodes: 78 Elapsed Time: 00:08:05
Gen 14 Best Fitness: 0.2095, Max Nodes: 78 Elapsed Time: 00:08:08
Gen 15 Best Fitness: 0.2095, Max Nodes: 78 Elapsed Time: 00:08:09
Gen 16 Best Fitness

In [17]:
best_net = pop.networks[fitnesses.index(best_fitness)].network
train = fitness(proc, best_net, X_train, y_train_encoded)
val = fitness(proc, best_net, X_val, y_val_encoded)

In [33]:
train

0.3030821917808219

In [18]:
print(best_net)

{
  "Associated_Data": {},
  "Edges": [
    {
      "from": 83,
      "to": 83,
      "values": [
        0.9615703425280682,
        1.0
      ]
    },
    {
      "from": 36,
      "to": 8,
      "values": [
        0.5800989362364866,
        4.0
      ]
    },
    {
      "from": 0,
      "to": 3,
      "values": [
        0.9627380347723928,
        2.0
      ]
    },
    {
      "from": 6,
      "to": 36,
      "values": [
        -0.6508994781530694,
        4.0
      ]
    },
    {
      "from": 95,
      "to": 68,
      "values": [
        0.3046924144273373,
        2.0
      ]
    },
    {
      "from": 36,
      "to": 2,
      "values": [
        0.7637230129362371,
        4.0
      ]
    },
    {
      "from": 36,
      "to": 67,
      "values": [
        0.5162342697186941,
        2.0
      ]
    },
    {
      "from": 94,
      "to": 118,
      "values": [
        -0.8391949873974536,
        4.0
      ]
    },
    {
      "from": 38,
      "to": 78,
      "values": [


In [21]:
print(best_net.num_edges())
print(best_net.num_nodes())

50
48


In [ ]:
type(best_net)

neuro.Network

In [ ]:
def read_network(fn):
    with open(fn, 'r') as f:
        s = f.read()
        j = json.loads(s)
        net = neuro.Network()
        net.from_json(j)
    return net

In [62]:
network2 = read_network('model.json')

In [ ]:
type(network2)

neuro.Network

In [41]:
get_prediction(proc, X_test[0])

RuntimeError: risp::Processor::get_risp_network() network_id 0 does not exist